<a href="https://colab.research.google.com/github/cheriexu/brain-networks-course/blob/master/psych162finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
#This is how we are going to do our project, colab gives us GPU usage up to 12 hr for free
!pip install numpy pandas matplotlib nibabel nilearn scipy sklearn keras

In [93]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [94]:
cd /content/drive/'My Drive'/PSYCH162

/content/drive/My Drive/PSYCH162


In [0]:
import pandas as pd
df = pd.read_csv("sampled_pheno.csv") #We have phenotypical data here too 


In [7]:
df.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'SUB_ID', 'X',
       'subject', 'SITE_ID', 'FILE_ID', 'DX_GROUP', 'DSM_IV_TR',
       ...
       'qc_notes_rater_1', 'qc_anat_rater_2', 'qc_anat_notes_rater_2',
       'qc_func_rater_2', 'qc_func_notes_rater_2', 'qc_anat_rater_3',
       'qc_anat_notes_rater_3', 'qc_func_rater_3', 'qc_func_notes_rater_3',
       'SUB_IN_SMP'],
      dtype='object', length=107)

## Get data: do not run again

In [0]:
#df_small = df[df].sample(200)

In [0]:
# https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/alff/'FILEIDHERE'.nii.gz
#Example URL
#!wget 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/func_minimal/Pitt_0050003_func_minimal.nii.gz'
#ITERATE THROUGH and then unzip (1112)

#DON'T RUN THIS AGAIN, ALREADY COLLECTED

'''for file_id in df_small['FILE_ID']:
  firstUrl = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/rois_ho/'
  filename = file_id+'_rois_ho.1D'
  !wget "$firstUrl$filename"
  #!gunzip $filename'''

In [0]:
#df_small.to_csv('sampled_pheno.csv',encoding='utf-8')

## Compute Correlational Matrices for Subjects

In [96]:

from nilearn.connectome import ConnectivityMeasure
import os
import numpy as np
directory = os.fsencode('.')
connectome_measure = ConnectivityMeasure(kind = 'correlation')
correlations = []
corr_org = []
files = []
for file in os.listdir(directory):
  #print(file)
  filename = os.fsdecode(file)
  if filename.endswith(".1D"):
    files.append(filename)
    df = pd.read_csv(filename,sep='\t')
    corr = connectome_measure.fit_transform([df.as_matrix()])
    corr_org.append(corr)
    corr = np.reshape(corr,(111,111))
    correlations.append(corr)
print(len(files))

184


## Get Phenotypic Dara

In [97]:
df = pd.read_csv("sampled_pheno.csv") #We have phenotypical data here too 
file_id = [file[0:file.index('_rois_ho.1D')] for file in files]
df = df.loc[df['FILE_ID'].isin(file_id)]
print(df.columns)
diagnoses_group = [df.loc[df['FILE_ID'] == f]['DX_GROUP'].values[0] for f in file_id]
pdd_category = [df.loc[df['FILE_ID'] == f]['DSM_IV_TR'].values[0] for f in file_id]
adi_r_social = [df.loc[df['FILE_ID'] == f]['ADI_R_SOCIAL_TOTAL_A'].values[0] for f in file_id]

adi_r_verbal= [df.loc[df['FILE_ID'] == f]['ADI_R_VERBAL_TOTAL_BV'].values[0] for f in file_id]
ados_total = [df.loc[df['FILE_ID'] == f]['ADOS_TOTAL'].values[0] for f in file_id]
ados_serverity = [df.loc[df['FILE_ID'] == f]['ADOS_GOTHAM_SEVERITY'].values[0] for f in file_id]
ados_socaffect = [df.loc[df['FILE_ID'] == f]['ADOS_GOTHAM_SOCAFFECT'].values[0] for f in file_id]

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'SUB_ID', 'X',
       'subject', 'SITE_ID', 'FILE_ID', 'DX_GROUP', 'DSM_IV_TR',
       ...
       'qc_notes_rater_1', 'qc_anat_rater_2', 'qc_anat_notes_rater_2',
       'qc_func_rater_2', 'qc_func_notes_rater_2', 'qc_anat_rater_3',
       'qc_anat_notes_rater_3', 'qc_func_rater_3', 'qc_func_notes_rater_3',
       'SUB_IN_SMP'],
      dtype='object', length=107)


*   DX_GROUP Diagnostic Group Numeric 1=Autism; 2=Control 1 2 2 DX_GROUP
*   PDD_DSM_IV_TR DSM-IV-TR PDD Category Numeric 0=none; 1=Autism; 2=Aspergers; 3=PDD-NOS 0 3 3 DSM_IV_TR
* ASD_DSM_5 DSM-5 ASD Category Numeric 0=none; 1=ASD 0 1 NA 
* ADI_R_SOCIAL_TOTAL_A Autism Diagnsoitc Interview-Revised Reciprocal Social Interaction Subscore 
* ADI_R_VERBAL_TOTAL_BV Abnormalities in Communication Subscore Verbal B(V) Total
* ADI_R_NONVERBAL_TOTAL_BV Abnormalities in Communication Subscore Nonverbal B(NV) Total 
* ADI_R_RRB_TOTAL_C Restricted, Repetitive, and Sterestyped Patterns of Behavior (C) Total
* ADOS_G_TOTAL ADOS Generic or ADOS-2 Module 4 Total Score
* ADOS_2_TOTAL ADOS-2 Total Score
* ADOS_2_SOCAFFECT ADOS-2 Social Affect Total 
* ADOS_2_TOTAL ADOS-2 Total Score
* ADOS_2_SEVERITY_TOTAL ADOS-2 Calibrated Severity Total Score 

http://fcon_1000.projects.nitrc.org/indi/abide/ABIDEII_Data_Legend.pdf

 

### K means Clustering Code

### Dunn indices (defunct)

In [0]:
from sklearn.metrics.pairwise import euclidean_distances


def normalize_to_smallest_integers(labels):
    """Normalizes a list of integers so that each number is reduced to the minimum possible integer, maintaining the order of elements.
    :param labels: the list to be normalized
    :returns: a numpy.array with the values normalized as the minimum integers between 0 and the maximum possible value.
    """

    max_v = len(set(labels)) if -1 not in labels else len(set(labels)) - 1
    sorted_labels = np.sort(np.unique(labels))
    unique_labels = range(max_v)
    new_c = np.zeros(len(labels), dtype=np.int32)

    for i, clust in enumerate(sorted_labels):
        new_c[labels == clust] = unique_labels[i]

    return new_c


def dunn(labels, distances):
    """
    Dunn index for cluster validation (the bigger, the better)
    
    .. math:: D = \\min_{i = 1 \\ldots n_c; j = i + 1\ldots n_c} \\left\\lbrace \\frac{d \\left( c_i,c_j \\right)}{\\max_{k = 1 \\ldots n_c} \\left(diam \\left(c_k \\right) \\right)} \\right\\rbrace
    
    where :math:`d(c_i,c_j)` represents the distance between
    clusters :math:`c_i` and :math:`c_j`, given by the distances between its
    two closest data points, and :math:`diam(c_k)` is the diameter of cluster
    :math:`c_k`, given by the distance between its two farthest data points.
    
    The bigger the value of the resulting Dunn index, the better the clustering
    result is considered, since higher values indicate that clusters are
    compact (small :math:`diam(c_k)`) and far apart.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    
    .. [Kovacs2005] Kovács, F., Legány, C., & Babos, A. (2005). Cluster validity measurement techniques. 6th International Symposium of Hungarian Researchers on Computational Intelligence.
    """

    labels = normalize_to_smallest_integers(labels)

    unique_cluster_distances = np.unique(min_cluster_distances(labels, distances))
    max_diameter = max(diameter(labels, distances))

    if np.size(unique_cluster_distances) > 1:
        return unique_cluster_distances[1] / max_diameter
    else:
        return unique_cluster_distances[0] / max_diameter


def min_cluster_distances(labels, distances):
    """Calculates the distances between the two nearest points of each cluster.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    """
    labels = normalize_to_smallest_integers(labels)
    n_unique_labels = len(np.unique(labels))

    min_distances = np.zeros((n_unique_labels, n_unique_labels))
    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i + 1, len(labels)):
            if labels[i] != labels[ii] and distances[i, ii] < min_distances[labels[i], labels[ii]]:
                min_distances[labels[i], labels[ii]] = min_distances[labels[ii], labels[i]] = distances[i, ii]
    return min_distances


def diameter(labels, distances):
    """Calculates cluster diameters (the distance between the two farthest data points in a cluster)
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :returns:
    """
    labels = normalize_to_smallest_integers(labels)
    n_clusters = len(np.unique(labels))
    diameters = np.zeros(n_clusters)

    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i + 1, len(labels)):
            if labels[i] == labels[ii] and distances[i, ii] > diameters[labels[i]]:
                diameters[labels[i]] = distances[i, ii]
    return diameters

### Start K means

In [0]:
from matplotlib import pyplot as plt 
from sklearn import cluster
from nilearn import plotting
from sklearn import metrics
num_cluster = [7]
clustering = []
for n in num_cluster:
  count = 0
  
  for corr in correlations:
    title = 'Original Correlation matrix with '+str(n)
    
    results = cluster.KMeans(n_clusters=n).fit_predict(corr)
    clustering.append(results)
 
    '''if count %20 == 0:
      display = plotting.plot_matrix(corr, vmax=1, vmin=-1,
                                 colorbar=True, title=title)
      tmp = corr[np.argsort(results),:]
      clustered = tmp[:,np.argsort(results)]
      title2 = 'Clustered Correlation matrix with '+str(n) 
      display = plotting.plot_matrix(clustered, vmax=1, vmin=-1,
                                 colorbar=True, title=title2)'''
    count+=1
  #silhouette = [metrics.silhouette_score(correlations[i], clustering[i]) for i in range(len(correlations))]
  #ch_score = [metrics.calinski_harabaz_score(correlations[i], clustering[i]) for i in range(len(correlations))]
  #d=  dunn(clustering[0], euclidean_distances(correlations[0]))
  #print(d)
  #print("Silhouette Coefficient for ",n ,' clusters',np.array(silhouette))
  #print("Dunn index for ",n ,' clusters',np.mean(np.array(d)))




In [0]:
intra_cluster_corr = []
for c in range(len(clustering)):
  cluster_corr = []
  for idx in range(num_cluster[0]):
    corr = correlations[c]
    cluster = clustering[c]
    corrmatrix = np.corrcoef(corr[np.where(cluster == idx)])
    upper = np.triu_indices(corrmatrix.shape[0], 1)
    cluster_corr.append(np.mean(corrmatrix[upper]))
  intra_cluster_corr.append(cluster_corr)
    
regressors = np.array(intra_cluster_corr)



## Regression

In [226]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.svm import SVR

logisticRegr = SVC(gamma = 'auto')#LogisticRegression()
logisticRegr.fit(regressors, np.array(diagnoses_group))
logisticRegr.predict(regressors)
score = logisticRegr.score(regressors,np.array( diagnoses_group))
print("For Diagnoses prediction", score)

lrmt = SVC(gamma = 'auto')#= LogisticRegression(multi_class='multinomial', solver = 'saga')
y = np.array(pdd_category)
lrmt.fit(regressors,y)
lrmt.predict(regressors)
score = lrmt.score(regressors,y)
print ('PDD Category from DSM IV ', score)

lr = LinearRegression()
y = np.array(adi_r_social)
y[np.where(y == -9.999e+03 )] = float('nan')
x = regressors[np.where(~np.isnan(y))]
y = y[ np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("adi-r-social ", score)

y= np.array(adi_r_verbal)
y[np.where(y == -9.999e+03 )] = float('nan')
x = regressors[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("adi-r-verbal ", score)

y= np.array(ados_total)
y[np.where(y == -9.999e+03 )] = float('nan')
x = regressors[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados total ", score)

y= np.array(ados_serverity)
y[np.where(y == -9.999e+03 )] = float('nan')
x = regressors[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados serverity ", score)

y= np.array(ados_socaffect)
y[np.where(y == -9.999e+03 )] = float('nan')
x = regressors[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados social affect ", score)



For Diagnoses prediction 0.5760869565217391
PDD Category from DSM IV  0.4891304347826087
adi-r-social  0.27180591929103937
adi-r-verbal  0.25633060666040164
ados total  0.1053232406272614
ados serverity  0.1874748108353732
ados social affect  0.22375285008442047


## Results

Classification accuracy:

* For Diagnoses prediction 0.5760869565217391
* PDD Category from DSM IV  0.4891304347826087

Regression Correlation:
* adi-r-social  0.27180591929103937
* adi-r-verbal  0.25633060666040164
* ados total  0.1053232406272614
* ados serverity  0.1874748108353732
* ados social affect  0.22375285008442047




Create cross correlation matrix across all time series for each subject \\
Store data in pandas dataframe \\
Use kmeans to cluster for baseline \\
Regress with psychometric data \\
Question: whether to use ALFF or minimally processed data\\
VAE and K means loss 


#VAE Clustering

In [105]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

from scipy.stats import norm

import keras
from keras import layers
from keras.models import Model
from keras import metrics
from keras import backend as K   # 'generic' backend so code works with either tensorflow or theano

K.clear_session()

np.random.seed(237)

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:4900: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))


In [0]:
img_shape = (111, 111,1)    
batch_size = 6
latent_dim = 7# Number of latent dimension parameters

# Encoder architecture: Input -> Conv2D*4 -> Flatten -> Dense
input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3,
                  padding='same', 
                  activation='relu')(input_img)
x = layers.Conv2D(64, 3,
                  padding='same', 
                  activation='relu',
                  strides=(3, 3))(x)
x = layers.Conv2D(64, 3,
                  padding='same', 
                  activation='relu')(x)
x = layers.Conv2D(64, 3,
                  padding='same', 
                  activation='relu')(x)
# need to know the shape of the network here for the decoder
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

# Two outputs, latent mean and (log)variance
z_mu = layers.Dense(latent_dim)(x)
z_log_sigma = layers.Dense(latent_dim)(x)

In [0]:
# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

In [0]:
# decoder takes the latent distribution sample as input
decoder_input = layers.Input(K.int_shape(z)[1:])

x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activation='relu')(decoder_input)

# reshape
x = layers.Reshape(shape_before_flattening[1:])(x)

# use Conv2DTranspose to reverse the conv layers from the encoder
x = layers.Conv2DTranspose(32, 3,
                           padding='same', 
                           activation='relu',
                           strides=(3, 3))(x)
x = layers.Conv2D(1, 3,
                  padding='same', 
                  activation='sigmoid')(x)

# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
z_decoded = decoder(z)

In [0]:
# construct a custom layer to calculate the loss
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        # Reconstruction loss
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    # adds the custom loss to the class
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

# apply the custom loss to the input images and the decoded latent distribution sample
y = CustomVariationalLayer()([input_img, z_decoded])

In [232]:
# VAE model statement
vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 111, 111, 1)  0                                            
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 111, 111, 32) 320         input_20[0][0]                   
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 37, 37, 64)   18496       conv2d_50[0][0]                  
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 37, 37, 64)   36928       conv2d_51[0][0]                  
__________________________________________________________________________________________________
conv2d_53 

In [233]:
corr_reshape = [np.reshape(c,(111,111,1)) for c in corr_org]
vae.fit(x=np.array(corr_reshape), y=None,
        shuffle=True,
        epochs=100,
        batch_size=8)


Epoch 1/100
184/184 [==============================] - 3s 15ms/step - loss: 2338326850693.7549
Epoch 2/100
184/184 [==============================] - 1s 4ms/step - loss: -0.2159
Epoch 3/100
184/184 [==============================] - 1s 4ms/step - loss: -0.3172
Epoch 4/100
184/184 [==============================] - 1s 4ms/step - loss: -0.3927
Epoch 5/100
184/184 [==============================] - 1s 4ms/step - loss: -0.4338
Epoch 6/100
184/184 [==============================] - 1s 4ms/step - loss: -0.4678
Epoch 7/100
184/184 [==============================] - 1s 4ms/step - loss: -0.4826
Epoch 8/100
184/184 [==============================] - 1s 4ms/step - loss: -0.4916
Epoch 9/100
184/184 [==============================] - 1s 4ms/step - loss: -0.4967
Epoch 10/100
184/184 [==============================] - 1s 4ms/step - loss: -0.5046
Epoch 11/100
184/184 [==============================] - 1s 4ms/step - loss: -0.5037
Epoch 12/100
184/184 [==============================] - 1s 4ms/step - los

In [234]:
encoder = Model(input_img, z_mu)
vae_reg = encoder.predict(np.array(corr_reshape), batch_size=batch_size)
'''print(x_valid_noTest_encoded[0])
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = plt.axes(projection='3d')
plt.figure(figsize=(10, 10))
ax.scatter3D(x_valid_noTest_encoded[:, 0], x_valid_noTest_encoded[:, 1],  x_valid_noTest_encoded[:, 2])
plt.show()'''
#Visualize latent space

"print(x_valid_noTest_encoded[0])\nfrom mpl_toolkits.mplot3d import Axes3D\nfig = plt.figure()\nax = plt.axes(projection='3d')\nplt.figure(figsize=(10, 10))\nax.scatter3D(x_valid_noTest_encoded[:, 0], x_valid_noTest_encoded[:, 1],  x_valid_noTest_encoded[:, 2])\nplt.show()"

In [235]:
from sklearn.svm import SVC
from sklearn.svm import SVR
lr = SVC(gamma='auto')
#logisticRegr = LogisticRegression()
vae_reg = np.array(vae_reg)
#print(vae_reg[np.where(np.isnan(vae_reg))])
lr.fit(vae_reg, np.array(diagnoses_group))

lr.predict(vae_reg)
score = lr.score(vae_reg,np.array( diagnoses_group))
print("For Diagnoses prediction", score)

#lrmt = LogisticRegression(multi_class='multinomial', solver =  'lbfgs')
y = np.array(pdd_category)
lr.fit(vae_reg,y)
lr.predict(vae_reg)
score = lr.score(vae_reg,y)
print ('PDD Category from DSM IV ', score)

#lr = LinearRegression()#LogisticRegression(multi_class='multinomial', solver =  'lbfgs')
lr = SVR()
y = np.array(adi_r_social)
y[np.where(y == -9.999e+03 )] = float('nan')
x = vae_reg[np.where(~np.isnan(y) )]
y = y[ np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("adi-r-social ", score)

y= np.array(adi_r_verbal)
y[np.where(y == -9.999e+03 )] = float('nan')
x = vae_reg[np.where(~np.isnan(y)  )]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("adi-r-verbal ", score)

y= np.array(ados_total)
y[np.where(y == -9.999e+03 )] = float('nan')
x = vae_reg[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados total ", score)

y= np.array(ados_serverity)
y[np.where(y == -9.999e+03 )] = float('nan')
x = vae_reg[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados serverity ", score)

y= np.array(ados_socaffect)
y[np.where(y == -9.999e+03 )] = float('nan')
x = vae_reg[np.where(~np.isnan(y))]
y = y[np.where(~np.isnan(y))]
lr.fit(x, y)
score = lr.score(x,y)
print("ados social affect ", score)

For Diagnoses prediction 0.8804347826086957
PDD Category from DSM IV  0.7119565217391305
adi-r-social  0.24082670670905074
adi-r-verbal  0.245677496541673
ados total  0.3118264475434387
ados serverity  0.3675975660126283
ados social affect  0.22979608027985154


## Results with VAE

With 7 latent variables: 

Classification Accuracy
* For Diagnoses prediction 0.8804347826086957
* PDD Category from DSM IV  0.7119565217391305

Correlation from Regression

* adi-r-social  0.24082670670905074
* adi-r-verbal  0.245677496541673
* ados total  0.3118264475434387
* ados serverity  0.3675975660126283
* ados social affect  0.22979608027985154



With 20 Latent Variables:

* For Diagnoses prediction 0.7934782608695652 (Classification Accuracy)
* PDD Category prediction from DSM IV  0.6793478260869565 (Classification Accuracy)

These are correlation
* adi-r-social  0.20724689666450424
* adi-r-verbal  0.1942414422201446
* ados total  0.1870398614136629
* ados serverity  0.3316804257464101
* ados social affect  0.21155238252912467